# **SytSim Example Usage Notebook**

This module enables stochastic and deterministic simulation of Markov Chain models of synaptic vesicle (SV) fusion under flexible model specifications.

**Contents:**
1. Model definition
2. Aggregate model definition (fusion clamps)
3. Simulation
4. Notes

In [4]:
import numpy as np
import pandas as pd
import multiprocessing as mp
import os
from markov_model import read_json_model, MarkovModel
from clamping import clamp_model_builder
import plotly.express as px

## **1. Model Definition**

Model are specified from dicts which can be defined as shown in the model_definition notebook, or loaded directly from a json file. These models are defined through the following fields:
1. *rate_matrix*: 2D array. Each element is a number or a string expression of parameter and stimulus names.
2. *parameters*: Dict. Parameter values (and units, for record keeping, not actually used). 
3. *initial_condition*: Numerical list specifying initial quantity / probability for each state.
4. *state_names*: (Optional) List of names for each state.

Model files / specifications may also contain other optional metadata.

In [5]:
# Load model specification from file and build MarkovModel from it
allosteric_file = os.path.join('models','allosteric_release.json')
allosteric_spec = read_json_model(allosteric_file)
allosteric_model = MarkovModel(**allosteric_spec)

In [6]:
# OR build MarkovModel directly from file
allosteric_file = os.path.join('models','allosteric_release.json')
allosteric_model = MarkovModel().import_model(allosteric_file)

## **2. Aggregate Model Definition**

This module allows you to build custom aggregate SV fusion models from a collection of individual Ca2+-sensor models using *clamp_model_builder* with these parameters:
1. *architecture*: A list of clamp details. For each clamp, provide a list of model specification dicts (as described above) for each sensor on that clamp.
2. *fusion_model*: (Optional) None or String specifying the fusion model from ['clamp_exponential', 'clamp_exponential_with_resistance']. Default: 'clamp_exponential'.
3. *replenishment_model*: (Optional) None or String specifying the SV replenishment model from ['constant']. Default: None.

Notes:
 - The 'clamp_exponential' models assume that each sensor has a state with name beginning with 'I' which indicates that sensor is 'inserted'. If all the sensors associated with a clamp are inserted then the clamp is 'unclamped'. If at least one sensor is not inserted then the clamp is 'clamped'. The rate of moving to the 'Fused' state depends exponentially on the number of unclamped clamps according to the Arrhenius equation.
 - Sensor models can be easily defined and edited in json files, and complex architectures can be easily built. New fusion and replenishment models require direct editing of the relevant functions in *clamping*.

In [7]:
from clamping import clamp_model_builder

In [8]:
# Get Synaptotagmin model specs from files
syt1_file = os.path.join('models','synaptotagmin_1.json')
syt1 = read_json_model(syt1_file)

syt7_file = os.path.join('models','synaptotagmin_7.json')
syt7 = read_json_model(syt7_file)

In [9]:
architecture = [
    [syt1],
    [syt1, syt1, syt7]
]

The above example describes an architecture where SV fusion is regulated by two clamps. The first is associated with a single synaptotagmin-1 sensor while the second is associated with two synaptotagmin-1s and a synaptotagmin-7.

Below we define a more realistic architecture. The SV is associated with 6 clamps, each associated with both a syt1 and a syt7 sensor. We go on to use the default fusion and replenishment rates, and build a MarkovModel for simulation

In [10]:
# Define clamp model
n_snares = 6
architecture = [[syt1, syt7] for _ in range(n_snares)]
fusion_model = 'clamp_exponential'
replenishment_model = None

# Build clamp model specifications
clamp_model_spec = clamp_model_builder(
    architecture=architecture,
    fusion_model=fusion_model,
    replenishment_model=replenishment_model
)

# Define as Markov Model for simulations
clamp_model = MarkovModel(**clamp_model_spec, name='Syt1 - Syt7')

## **3. Simulation**

MarkovModels are simulated in response to arbitrary stimuli through the *simulate* class method. The following name-value pairs are expected by the method, which are optional and what is required depends on the nature of the model and simulation:

1. *stimuli*: Dict of stimuli names and values. Multiple stimuli can be specified in the case that transition rates depend on multiple signals (e.g. 'v' and 'ca' for combined voltage & calcium stimulation). Each stimulus name is mapped to a dict which must contain 'timestamp' and 'value' but can contain other data such as units for record keeping.
2. *timestamp*: Outputs interpolated to align with these timepoints. If not provided then this is taken from the stimulus with the shortest duration.
3. *mode*: 'deterministic' or 'stochastic'. (See section **4. Notes**) Default: 'stochastic'.

For 'stochastic' models, specify either:
1. *n_simulations*: Int. How many stochastic simulations to perform. Default: 1.
2. *n_events_required*: Dict: {state(Str): n_events(Int)}. Continue simulations until the required number of events for each specified state is achieved.

Providing 'n_simulations' will overwrite 'n_events_required'. The following optional parameters are expected for stochastic simulations:
1. *batch_size*: Int. If *n_events_required* is provided then simulations are run in batches of this size until requirements are met. Otherwise *batch_size* is set equal to *n_simulations* and only one batch is run. Default: 10000.
2. *n_processes*: Int (*stochastic* only). How many parallel processes to use. Default: 1.
3. *record*: List of state names to record (*stochastic* only). For each state in record the time it is jumped into is saved and aggregated across all simulations. Processed results like cumulative probability and event rate are also calculated for each. Default: all states are recorded.
4. *track_clamps*: Bool (*stochastic* only). Toggle to calculate the average number of free ('unclamped') clamps over time. Requires 'i_free_clamps' to be in *record* for i = 0 to n_clamps. Can slightly reduce computational time. Default: False.

In the example below the allosteric and Syt1 - Syt7 models are simulated stochastically and deterministically in response to 50 uM steps in [Ca2+].

In [11]:
# Define stimulus
t_end = 1
ca_timestamp = np.linspace(0, t_end, 2)
ca_step = [50] * len(ca_timestamp)
stimuli = {
    'ca' : {
        'timestamp' : ca_timestamp,
        'value' : ca_step,
        'time_units' : 'ms',
        'stim_units' : 'uM'
    }
}

# Request outputs at specified times
timestamp = np.linspace(0, t_end, 1000)

In [12]:
# Simulate allosteric model
allosteric_sim = allosteric_model.simulate(
    stimuli=stimuli,
    mode='deterministic',
    record=['Fused'],
    timestamp=timestamp
)

# Simulate clamp model 1000 times
clamp_sim = clamp_model.simulate(
    stimuli=stimuli,
    mode='stochastic',
    record=['Fused'],
    n_simulations=1000,
    n_processes=mp.cpu_count(),
    timestamp=timestamp
)

# Simulate clamp model until 1000 'Fused' events occur and record the release of clamps over time.
clamp_sim_alt = clamp_model.simulate(
    stimuli=stimuli,
    mode='stochastic',
    record=['Fused'] + [f"{i}_free_clamps" for i in range(n_snares+1)],
    track_clamps=True,
    n_processes=mp.cpu_count(),
    n_events_required={'Fused': 1000},
    batch_size=100,
    timestamp=timestamp
)

In [13]:
# Visualise cumulative probability of fusion
allo_col = 'rgb(128,128,128)'
clamp_col = 'rgb(255,167,0)'

fig = px.line()

fig.add_scatter(x=allosteric_sim['timestamp'], y=allosteric_sim['probability']['Fused'],
mode='lines', line={'color':allo_col, 'dash':'dash'}, name='Allosteric')

fig.add_scatter(x=clamp_sim['timestamp'], y=clamp_sim['probability']['Fused'],
mode='lines', line={'color':clamp_col, 'dash':'solid'}, name=clamp_model.name)
fig.add_scatter(x=clamp_sim_alt['timestamp'], y=clamp_sim_alt['probability']['Fused'],
mode='lines', line={'color':'black', 'dash':'dot'}, name=clamp_model.name)

fig.update_layout(
    legend = {'title': 'Legend'},
    xaxis = {'title': 'Time (ms)', 'range': [0, 1]},
    yaxis = {'title': 'Cum. Fusion Prob.', 'range': [0, 1]},
    hovermode='x'
)

fig.show()

In [14]:
allo_col = 'rgb(128,128,128)'
clamp_col = 'rgb(255,167,0)'

fig = px.line()
fig.add_scatter(x=clamp_sim_alt['timestamp'], y=clamp_sim_alt["free_clamps"],
mode='lines', line={'color':clamp_col, 'dash':'solid'}, name=clamp_model.name)
fig.update_layout(
    legend = {'title': 'Legend'},
    xaxis = {'title': 'Time (ms)', 'range': [0, 1]},
    yaxis = {'title': 'Mean Free Clamps', 'range': [0, 4]},
    hovermode='x'
)
fig.show()

fig = px.line()
fig.add_scatter(x=clamp_sim_alt['timestamp'], y=clamp_sim_alt["free_clamps_fused"]["mean"],
mode='lines', line={'color':clamp_col, 'dash':'dash'}, name=clamp_model.name)
fig.update_layout(
    legend = {'title': 'Legend'},
    xaxis = {'title': 'Time (ms)', 'range': [0, 1]},
    yaxis = {'title': ' Mean free clamps on fused SVs', 'range': [0, 4]},
    hovermode='x'
)
fig.show()

fig = px.line()
fig.add_scatter(x=clamp_sim_alt['timestamp'], y=clamp_sim_alt["free_clamps_unfused"],
mode='lines', line={'color':clamp_col, 'dash':'solid'}, name=clamp_model.name)
fig.update_layout(
    legend = {'title': 'Legend'},
    xaxis = {'title': 'Time (ms)', 'range': [0, 1]},
    yaxis = {'title': 'Mean Free Clamps on unfused SVs'},
    hovermode='x'
)
fig.show()

### **3.1 Action Potential Simulation**

In [15]:
# Action potential simulation:
ap_file = os.path.join('calcium','2AP_40nm_20ms.csv')
ap_data = pd.read_csv(ap_file, header=None)
ap_stimuli = {
    'ca' : {
        'timestamp' : np.array(ap_data[0]),
        'value' : np.array(ap_data[1]),
        'time_units' : 'ms',
        'stim_units' : 'uM'
    }
}

# Simulate allosteric model
allosteric_ap_sim = allosteric_model.simulate(
    stimuli=ap_stimuli,
    mode='deterministic',
    record=['Fused']
)

# Simulate clamp model
clamp_ap_sim = clamp_model.simulate(
    stimuli=ap_stimuli,
    mode='stochastic',
    record=['Fused'],
    n_simulations=1000,
    n_processes=mp.cpu_count(),
)


In [16]:
fig = px.line()

fig.add_scatter(x=allosteric_ap_sim['timestamp'], y=allosteric_ap_sim['probability']['Fused'],
mode='lines', line={'color':allo_col, 'dash':'dash'}, name='Allosteric')

fig.add_scatter(x=clamp_ap_sim['timestamp'], y=clamp_ap_sim['probability']['Fused'],
mode='lines', line={'color':clamp_col, 'dash':'solid'}, name=clamp_model.name)

fig.update_layout(
    legend = {'title': 'Legend'},
    xaxis = {'title': 'Time (ms)', 'range': [0, 25]},
    yaxis = {'title': 'Cum. Fusion Prob.', 'range': [0, 0.25]},
    hovermode='x'
)

fig.show()

## **4. Notes**

In order to complete a deterministic simulation the model must be specified as a *rate_matrix* with a numerical *initial_condition*, as in the model files. Stochastic simulations require a sparse dictionary of non-zero *transitions* mapping *source* states to a list of destinations with transition rates. When a MarkovModel is initialised from a rate matrix it is automatically converted to a sparse *transitions* dict so both deterministic and stochastic models can be performed. The reverse is not true, so a model specified from sparse *transitions* for stochastic simulation is not automatically converted to a *rate_matrix* for deterministic simulation. When an aggregate clamp model is built it is defined as sparse *transitions* because the full rate matrices for these models are inconvenient to define from the individual sensors in conjunction with spontaneous state and rate update rules (e.g. clamp-dependent fusion). Consequently models built with the *clamp_model_builder* may only be simulated stochastically.